In [6]:
import numpy as np
import pandas as pd
from osgeo import gdal
import os
import time
from scipy.optimize import curve_fit

In [7]:
class Dataset:
    def __init__(self, in_file):
        self.in_file = in_file  # Tiff或者ENVI文件

        dataset = gdal.Open(self.in_file)
        self.XSize = dataset.RasterXSize  # 网格的X轴像素数量
        self.YSize = dataset.RasterYSize  # 网格的Y轴像素数量
        self.Bands = dataset.RasterCount  # 波段数
        self.GeoTransform = dataset.GetGeoTransform()  # 投影转换信息
        self.ProjectionInfo = dataset.GetProjection()  # 投影信息

#     def get_data(self, band):
#         #band: 读取第几个通道的数据
#         dataset = gdal.Open(self.in_file)
#         band = dataset.GetRasterBand(band)
#         data = band.ReadAsArray()
#         return data
    
    def get_data(self):
        #band: 读取第几个通道的数据
        dataset = gdal.Open(self.in_file)
        data = dataset.ReadAsArray(0,0,self.XSize,self.YSize)
        return data
    

    def get_lon_lat(self):
        #获取经纬度信息
        gtf = self.GeoTransform
        x_range = range(0, self.XSize)
        y_range = range(0, self.YSize)
        x, y = np.meshgrid(x_range, y_range)
        lon = gtf[0] + x * gtf[1] + y * gtf[2]
        lat = gtf[3] + x * gtf[4] + y * gtf[5]
        
        lon_lat=[]
        for (longitude,latitude) in zip(lon,lat):
            lon_lat.append(list(zip(longitude,latitude)))
            
        return np.array(lon_lat)
    
    def new_dataset(self,data,lon_lat):
        new_dataset=[]
        for i in range(self.YSize):
            for j in range(self.XSize):
                x1 = lon_lat[i,j]
                x2 = data[:,i,j]
                x=np.hstack((x1,x2))
                new_dataset.append(x)
#         for (i,j) in zip(lon_lat[i,j],data[:,i,j]):
#             new_dataset.append(list(zip(i,j)))
            
        return np.array(new_dataset)

In [8]:
def func(x, m1, m2, m3, m4, m5, m6):
    return m1 + m2 /(1 + np.exp(-m3 * (x-m4))) - m2/(1 + np.exp(-m5 * (x-m6)))

#注意初值
def DofittingandValidation(yData, xData):
#     p=[5,40,0.1,140,0.1,270]#lai
#     p=[1000,7000,0.1,140,0.1,270]#NDVI
    p=[900,5000,0.1,140,0.1,270]#EVI
    
    Parameters, pcov = curve_fit(func, xData, yData, p0=p, maxfev=1000000000)#,method='trf', maxfev=1000000)
    return Parameters

#注意时间序列
def get_python_param(data):
    timeDims=data.shape[1]
    xData = 16 * (np.array(range(timeDims))) + 9
#     xData=4 * (np.array(list(range(timeDims)))) + 1
    pars=[]
    for i in range(data.shape[0]):
        lon_lat=data_lon_lat[i]
        yData = data[i,:]
        pars_ = DofittingandValidation(yData, xData) 
        x=np.hstack((lon_lat,pars_))
        pars.append(x)
    return np.array(pars)

In [9]:
start_time = time.time()
dir_path = r"D:\Desktop\mypaper\data"
filename = "gee-EVI-605360.tif"
file_path = os.path.join(dir_path, filename)
dataset = Dataset(file_path)

data = dataset.get_data( )  # 获取第一个通道的数据

lon_lat = dataset.get_lon_lat()  # 获取经纬度信息longitude, latitude

new_dataset=dataset.new_dataset(data,lon_lat)

data_lon_lat=new_dataset[:,0:2]
data=new_dataset[:,2::]
param=get_python_param(data)
print("Thread pool execution in " + str(time.time() - start_time), "seconds")

d:\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Thread pool execution in 2819.288745164871 seconds


In [5]:
print(param.shape)

(605360, 8)


In [12]:
# 297571

LAI=5718.964570760727,40*60
NDVI=1318.1171004772186,27*60
EVI=1169.101529121399,18*60
print(LAI,NDVI,EVI)

(5718.964570760727, 2400) (1318.1171004772186, 1620) (1169.101529121399, 1080)


In [ ]:
# 605360
LAI=7711.796985864639,55*60
NDVI=2898.5893547534943,27*60
EVI=2819.288745164871,32*60